In [ ]:
import pandas as pd

# **Lectura de bases de datos con SQL desde un csv**

In [ ]:
# Importamos SQLite
import sqlite3 
# conectamos a la BD
db_con = sqlite3.connect(r'database.db') 
# Leemos el csv
o = pd.read_csv("https://raw.githubusercontent.com/DiegoCorderoBegueria/TheValley/main/orders.csv")
p = pd.read_csv("https://raw.githubusercontent.com/DiegoCorderoBegueria/TheValley/main/products.csv")
# Escribimos los csvs como tablas en la base de datos.
o.to_sql('orders', db_con, if_exists='replace', index=False) 
p.to_sql('products', db_con, if_exists='replace', index=False)

In [ ]:
#Con la función read_sql de pandas podemos introducir código SQL y extraer datos de una base de datos.
#Solo necesitamos introducir dos argumentos: la consulta SQL, y la conexión a la base de datos.
orders = pd.read_sql("SELECT * FROM orders", db_con)
products = pd.read_sql("SELECT * FROM products", db_con)
orders.head()

,Customer_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Product_ID,Quantity
0,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-LA-10004425,3
1,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-BI-10000320,3
2,DE-13255,CA-2014-109134,2014-11-05,2014-11-10,Standard Class,FUR-FU-10000320,6
3,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,FUR-FU-10001468,2
4,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,OFF-BI-10003684,2


In [ ]:
#Podemos introducir la consulta SQL tan compleja como queramos, hacer joins, quitar filas, columnas...
#DFJoin=pd.read_sql("SELECT t1.Segment,sum(Quantity*Price) as Amount FROM (SELECT Segment,Customer_ID FROM customers) as t1 JOIN (SELECT Customer_ID,Product_ID,Quantity FROM orders) AS t2 ON t1.Customer_ID=t2.Customer_ID JOIN (SELECT Product_ID,Price FROM products) as t3 ON t2.Product_ID=t3.Product_ID GROUP BY 1 ORDER BY 2 DESC",db_con)
order_and_products = pd.read_sql("SELECT * FROM orders JOIN products ON orders.Product_ID = products.Product_ID", db_con)
order_and_products.head()

,Customer_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Product_ID,Quantity,Product_ID,Category,Sub_Category,Product_Name,Price
0,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-LA-10004425,3,OFF-LA-10004425,Office Supplies,Labels,Staple-on labels,208.0
1,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-BI-10000320,3,OFF-BI-10000320,Office Supplies,Binders,GBC Plastic Binding Combs,78.0
2,DE-13255,CA-2014-109134,2014-11-05,2014-11-10,Standard Class,FUR-FU-10000320,6,FUR-FU-10000320,Furniture,Furnishings,OIC Stacking Trays,351.0
3,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,FUR-FU-10001468,2,FUR-FU-10001468,Furniture,Furnishings,Tenex Antistatic Computer Chair Mats,456.0
4,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,OFF-BI-10003684,2,OFF-BI-10003684,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,443.0


In [ ]:
#Cerramos la conexion
db_con.close()

# **Merge**

Merge junta o concatena dos dataframes, de la manera en que especifiquemos

In [ ]:
#Con sufijos dependientes de la tabla origen
orders_and_products = orders.merge(products, how="left", on="Product_ID", suffixes=("_orders", "_products"))
orders_and_products.head()

,Customer_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Product_ID,Quantity,Category,Sub_Category,Product_Name,Price
0,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-LA-10004425,3,Office Supplies,Labels,Staple-on labels,208.0
1,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-BI-10000320,3,Office Supplies,Binders,GBC Plastic Binding Combs,78.0
2,DE-13255,CA-2014-109134,2014-11-05,2014-11-10,Standard Class,FUR-FU-10000320,6,Furniture,Furnishings,OIC Stacking Trays,351.0
3,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,FUR-FU-10001468,2,Furniture,Furnishings,Tenex Antistatic Computer Chair Mats,456.0
4,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,OFF-BI-10003684,2,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,443.0


In [ ]:
#Si las tablas no tuvieran el mismo nombre de columna para la union
orders = orders.rename(columns={"Product_ID": "ID_Product"})
orders_and_products = orders.merge(products, how="left", left_on="ID_Product", right_on="Product_ID", suffixes=("_orders", "_products"))
orders_and_products.head()

,Customer_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,ID_Product,Quantity,Product_ID,Category,Sub_Category,Product_Name,Price
0,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-LA-10004425,3,OFF-LA-10004425,Office Supplies,Labels,Staple-on labels,208.0
1,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-BI-10000320,3,OFF-BI-10000320,Office Supplies,Binders,GBC Plastic Binding Combs,78.0
2,DE-13255,CA-2014-109134,2014-11-05,2014-11-10,Standard Class,FUR-FU-10000320,6,FUR-FU-10000320,Furniture,Furnishings,OIC Stacking Trays,351.0
3,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,FUR-FU-10001468,2,FUR-FU-10001468,Furniture,Furnishings,Tenex Antistatic Computer Chair Mats,456.0
4,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,OFF-BI-10003684,2,OFF-BI-10003684,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,443.0


# **Join**

Join junta o concatena dos dataframes, de la manera que especifiquemos

In [ ]:
orders = pd.read_csv("https://raw.githubusercontent.com/DiegoCorderoBegueria/TheValley/main/orders.csv")
products = pd.read_csv("https://raw.githubusercontent.com/DiegoCorderoBegueria/TheValley/main/products.csv")
#products = orders #Si queremos ver abajo los sufijos

In [ ]:
#Funciona muy parecido al Merge, pero requiere que la tabla derecha tenga asignado un index
orders.join(products.set_index("Product_ID"), on="Product_ID", lsuffix="_o", rsuffix="_p")

,Customer_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Product_ID,Quantity,Category,Sub_Category,Product_Name,Price
0,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-LA-10004425,3,Office Supplies,Labels,Staple-on labels,208.0
1,MG-17875,CA-2014-103744,2014-02-23,2014-02-27,Standard Class,OFF-BI-10000320,3,Office Supplies,Binders,GBC Plastic Binding Combs,78.0
2,DE-13255,CA-2014-109134,2014-11-05,2014-11-10,Standard Class,FUR-FU-10000320,6,Furniture,Furnishings,OIC Stacking Trays,351.0
3,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,FUR-FU-10001468,2,Furniture,Furnishings,Tenex Antistatic Computer Chair Mats,456.0
4,MD-17350,CA-2014-114251,2014-11-05,2014-11-10,Standard Class,OFF-BI-10003684,2,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,443.0
...,...,...,...,...,...,...,...,...,...,...,...
9989,DR-12880,CA-2017-161956,2017-08-27,2017-08-29,Second Class,OFF-ST-10001370,5,Office Supplies,Storage,Sensible Storage WireTech Storage Systems,851.0
9990,DR-12880,CA-2017-161956,2017-08-27,2017-08-29,Second Class,FUR-CH-10004886,3,Furniture,Chairs,Bevis Steel Folding Chairs,832.0
9991,DR-12880,CA-2017-161956,2017-08-27,2017-08-29,Second Class,OFF-BI-10003650,2,Office Supplies,Binders,GBC DocuBind 300 Electric Binding Machine,628.0
9992,DR-12880,CA-2017-161956,2017-08-27,2017-08-29,Second Class,OFF-PA-10001289,3,Office Supplies,Paper,White Computer Printout Paper by Universal,84.0


# **Map**

Map aplica una función X a todos los elementos de una columna de un dataframe

La función [Map()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html) toma como argumento otra función, que contiene el código que queremos aplicar a cada elemento de la lista o columna.

In [ ]:
#Funcion
def extrae_codigo(x):
  return x[0:3]

In [ ]:
products["Codigo_Categoría"] = products["Product_ID"].map(extrae_codigo)
products.head()

,Product_ID,Category,Sub_Category,Product_Name,Price,Codigo_Categoría
0,FUR-BO-10000112,Furniture,Bookcases,"Bush Birmingham Collection Bookcase, Dark Cherry",422.0,FUR
1,FUR-BO-10000330,Furniture,Bookcases,"Sauder Camden County Barrister Bookcase, Plank...",847.0,FUR
2,FUR-BO-10000362,Furniture,Bookcases,Sauder Inglewood Library Bookcases,828.0,FUR
3,FUR-BO-10000468,Furniture,Bookcases,O'Sullivan 2-Shelf Heavy-Duty Bookcases,722.0,FUR
4,FUR-BO-10000711,Furniture,Bookcases,"Hon Metal Bookcases, Gray",633.0,FUR


# **Apply**

La función Apply funciona de forma muy parecida a la funcion Map. Necesita una función que aplicará a todos los elementos, pero puede aplicarse tanto a una columna entera como a parte o todo el DataFrame.

In [ ]:
df = pd.DataFrame({ 'A': [1,2,3,4], 
                   'B': [10,20,30,40],
                   'C': [20,40,60,80]
                  }, 
                  index=['Row 1', 'Row 2', 'Row 3', 'Row 4'])

df

,A,B,C
Row 1,1,10,20
Row 2,2,20,40
Row 3,3,30,60
Row 4,4,40,80


In [ ]:
#Por defecto se aplica sobre las columnas del dataframe
df.apply(sum)

A     10
B    100
C    200
dtype: int64

In [ ]:
#Para aplicarlo sobre las filas necesitamos icluirle el parámetro axis=1
df.apply(sum, axis=1)

Row 1     31
Row 2     62
Row 3     93
Row 4    124
dtype: int64

In [ ]:
#Por ejemplo también le podemos pasar una función que resta dos columnas
def mi_funcion(x):
  return (x["B"] - x["A"])


df.apply(mi_funcion, axis=1)

Row 1     9
Row 2    18
Row 3    27
Row 4    36
dtype: int64

In [ ]:
#También podemos pasarle una lambda
products["Descuento"] = products.apply(lambda x: 0.12 if x["Sub_Category"]=="Bookcases" and x["Price"]>600 else 0 , axis=1)

products.head()

,Product_ID,Category,Sub_Category,Product_Name,Price,Codigo_Categoría,Descuento
0,FUR-BO-10000112,Furniture,Bookcases,"Bush Birmingham Collection Bookcase, Dark Cherry",422.0,FUR,0.00
1,FUR-BO-10000330,Furniture,Bookcases,"Sauder Camden County Barrister Bookcase, Plank...",847.0,FUR,0.12
2,FUR-BO-10000362,Furniture,Bookcases,Sauder Inglewood Library Bookcases,828.0,FUR,0.12
3,FUR-BO-10000468,Furniture,Bookcases,O'Sullivan 2-Shelf Heavy-Duty Bookcases,722.0,FUR,0.12
4,FUR-BO-10000711,Furniture,Bookcases,"Hon Metal Bookcases, Gray",633.0,FUR,0.12


# **Agregaciones (Group by, Agg...)**

Podemos agrupar el dataframe con la funcion **GroupBy**, de tal forma que las funciones posteriores se apliquen a cada grupo por separado.

En pandas hay que eliminar la agrupación al finalizar, si no el dataframe se mantiene agrupadoy nos dificulta su lectura y manejo. Para ello llamamos a la funcion reset_index()

In [ ]:
products.groupby(by=["Category", "Sub_Category"])["Price"].sum()

Category         Sub_Category
Furniture        Bookcases        29312.0
                 Chairs           43189.0
                 Furnishings      95106.0
                 Tables           27744.0
Office Supplies  Appliances       53338.0
                 Art              90172.0
                 Binders         101781.0
                 Envelopes        22793.0
                 Fasteners        19094.0
                 Labels           35505.0
                 Paper           136658.0
                 Storage          65629.0
                 Supplies         19864.0
Technology       Accessories      72253.0
                 Copiers           4802.0
                 Machines         32637.0
                 Phones           96387.0
Name: Price, dtype: float64

Con la función **agg** podemos hacer multiples agregaciones sobre una misma columna a un dataset agrupado, o incluso variar la agregación seleccionada en función de la columna.

In [ ]:
#Visualizamos el maximo de cada agrupación
products.groupby(["Category", "Sub_Category"]).agg(["max"])

Product_ID  ...   Price
                                          max  ...     max
Category        Sub_Category                   ...        
Furniture       Bookcases     FUR-BO-10004834  ...   944.0
                Chairs        FUR-CH-10004997  ...   992.0
                Furnishings   FUR-FU-10004973  ...   999.0
                Tables        FUR-TA-10004915  ...   975.0
Office Supplies Appliances    OFF-AP-10004980  ...   997.0
                Art           OFF-AR-10004999  ...   989.0
                Binders       OFF-BI-10004995  ...   996.0
                Envelopes     OFF-EN-10004955  ...   990.0
                Fasteners     OFF-FA-10004968  ...   908.0
                Labels        OFF-LA-10004853  ...   986.0
                Paper         OFF-PA-10004996  ...   997.0
                Storage       OFF-ST-10004963  ...   999.0
                Supplies      OFF-SU-10004884  ...   951.0
Technology      Accessories   TEC-AC-10004992  ...  1000.0
                Copiers       TEC-CO-10004722  ...   839.0
                Machines      TEC-MA-10004679  ...   986.0
                Phones        TEC-PH-10004977  ...   992.0

[17 rows x 3 columns]

In [ ]:
#Visualizamos el precio medio de las categorías y contamos los productos incluidos en cada una
products.groupby(["Category", "Sub_Category"]).agg({"Price": "mean", "Product_ID": "nunique"})

Price  Product_ID
Category        Sub_Category                        
Furniture       Bookcases     586.240000          49
                Chairs        490.784091          87
                Furnishings   505.882979         182
                Tables        486.736842          57
Office Supplies Appliances    538.767677          98
                Art           549.829268         163
                Binders       477.845070         210
                Envelopes     422.092593          54
                Fasteners     444.046512          43
                Labels        507.214286          70
                Paper         481.190141         276
                Storage       493.451128         131
                Supplies      522.736842          38
Technology      Accessories   491.517007         144
                Copiers       369.384615          13
                Machines      509.953125          63
                Phones        509.984127         184

In [ ]:
#Podemos incluir la cantidad de agrupaciones que queramos tanto por variables como por funciones
products.groupby(["Category", "Sub_Category"]).agg({"Price": ["max","min","mean"], "Product_ID": ["nunique"]}).reset_index()

Category Sub_Category   Price                   Product_ID
                                     max   min        mean    nunique
0         Furniture    Bookcases   944.0  64.0  586.240000         49
1         Furniture       Chairs   992.0  19.0  490.784091         87
2         Furniture  Furnishings   999.0   1.0  505.882979        182
3         Furniture       Tables   975.0   1.0  486.736842         57
4   Office Supplies   Appliances   997.0   5.0  538.767677         98
5   Office Supplies          Art   989.0   5.0  549.829268        163
6   Office Supplies      Binders   996.0   3.0  477.845070        210
7   Office Supplies    Envelopes   990.0  10.0  422.092593         54
8   Office Supplies    Fasteners   908.0   8.0  444.046512         43
9   Office Supplies       Labels   986.0   3.0  507.214286         70
10  Office Supplies        Paper   997.0   3.0  481.190141        276
11  Office Supplies      Storage   999.0  12.0  493.451128        131
12  Office Supplies     Supplies   951.0  40.0  522.736842         38
13       Technology  Accessories  1000.0   4.0  491.517007        144
14       Technology      Copiers   839.0  34.0  369.384615         13
15       Technology     Machines   986.0  16.0  509.953125         63
16       Technology       Phones   992.0   1.0  509.984127        184